In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score,accuracy_score

In [2]:
music = pd.read_csv('subsampled.csv')

In [5]:
music['Genre'].unique()

array(['Blues', 'Classical', 'EDM', 'Ghazal', 'Hip-Hop', 'Metal', 'Pop',
       'Qawwali', 'Rap', 'Rock'], dtype=object)

In [26]:
##Extracting all spotify features as a matrix
X = music.iloc[:, 7:15].values

## Extracting Genres
y = music['Genre']

# split into training data (80%) and test data (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Standardize
scaler = StandardScaler()
scaler.fit(X_train)
X_train_standardized = scaler.transform(X_train)
X_test_standardized = scaler.transform(X_test)

# train SVM
svm = SVC(kernel='linear', C=1.0, random_state=0)
svm.fit(X_train,y_train)
y_predicted = svm.predict(X_test)
# Confusion Matrix
print("Confusion Matrix:\n", confusion_matrix(y_true=y_test, y_pred=y_predicted))
## 
print("Accuracy :\n", accuracy_score(y_true=y_test, y_pred=y_predicted))


Confusion Matrix:
 [[ 3  2  2  2  0  0  1  1  5  1]
 [ 0 12  0  1  0  0  0  0  0  0]
 [ 2  0 11  0  0  2  0  1  1  1]
 [ 2  2  2 10  0  0  1  4  1  3]
 [ 4  0  4  1  0  1  1  1  5  0]
 [ 1  0  1  0  0  9  0  1  0  4]
 [ 3  0  6  1  0  2  0  2  2  0]
 [ 1  2  5  2  1  1  1  5  1  2]
 [ 1  2  1  1  2  0  0  0  7  0]
 [ 1  1  3  1  0  5  1  0  1  4]]
Accuracy :
 0.3505747126436782
